<a href="https://colab.research.google.com/github/prasannakumarbalam/FMML_Project_and_Labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

# New Section

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

# ***3)*** The percentage to reserve for the validation set in machine learning depends on various factors specific to your project:

1. Dataset Size: Larger datasets can allocate a smaller percentage to validation; smaller datasets may need a larger portion for validation.

2. Model Complexity: Complex models might require more validation data to accurately assess generalization, while simpler models may need less.

3. Resource Constraints: Consider computational resources; smaller validation sets can speed up training and evaluation.

4. Cross-Validation: Techniques like k-fold cross-validation adapt the validation set size based on the number of folds (k).

5. Domain Knowledge: Understand your data's characteristics; some datasets require larger validation sets due to inherent variability.

6. Trial and Error: Experiment with different validation set sizes to find the balance between training and validation that works best for your specific project.

Common starting points are 80/20 or 70/30 splits (training/validation), but adjust based on your project's needs and goals. The aim is to have a validation set large enough for reliable performance assessment while ensuring sufficient training data.

# ***2)*** The size of the training and validation sets in machine learning can significantly impact your ability to predict the accuracy of a model on the test set using the validation set:

1. A large training set can help the model learn better but may lead to less reliable validation set estimates if it's small.
2. A large validation set provides a more stable performance estimate but may not be sufficient for a small training set to learn effectively.
3. Balancing the sizes of the training and validation sets strikes a middle ground and often provides a reasonable compromise.
4. Cross-validation techniques like k-fold cross-validation can help mitigate issues related to dataset size and provide robust estimates of model performance.

The choice of set sizes should consider factors like dataset characteristics, model complexity, and available resources to strike the right balance between training and validation set sizes.

# ***1)*** The size of the validation set in machine learning can impact the accuracy assessment and performance of a model:

1. **Increasing Validation Set Size**:
   - Pros: More reliable performance estimate, better detection of overfitting.
   - Cons: Less data for training, potential underfitting, especially with limited data.

2. **Reducing Validation Set Size**:
   - Pros: More data for training, potentially higher model accuracy.
   - Cons: Less reliable performance estimate, harder to detect overfitting.

The choice of validation set size should consider dataset characteristics, project goals, and resource constraints. Techniques like k-fold cross-validation can provide a balance between validation set size and model reliability. The ideal size depends on your specific project context.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# ***3)*** In summary, the number of iterations during model training can impact the estimate of a model's performance:

1. **Too Few Iterations:** Insufficient iterations may lead to underfitting, causing the model to perform poorly. More iterations can help the model converge and potentially improve performance.

2. **Too Many Iterations:** Excessive iterations can result in overfitting, where the model fits noise in the data, leading to poor generalization. It's crucial to strike a balance to prevent overfitting.

3. **Hyperparameter Tuning:** The number of iterations is often a hyperparameter that needs tuning. Cross-validation can help determine the optimal number of iterations for a specific dataset and model complexity.

Ultimately, the impact of the number of iterations on the estimate of model performance depends on various factors, and careful experimentation and tuning are necessary to find the right balance.

# ***1)*** Sure, here are the key points about averaging validation accuracy across multiple splits using cross-validation:

1. **Consistency**: Cross-validation reduces variability in evaluation results by using multiple data splits.

2. **Generalization**: It provides a more comprehensive view of how well a model generalizes to unseen data.

3. **Reliable Performance**: Cross-validation offers a more robust estimate of a model's performance by considering different data partitions.

4. **Model Selection**: It helps in choosing the best model, hyperparameters, or feature sets by comparing performance across folds.

In essence, cross-validation enhances the reliability of model evaluation and aids in making informed decisions in machine learning.

# 2)Cross-validation, specifically k-fold cross-validation, provides a more accurate estimate of a model's performance on unseen data compared to a single train-test split. However, it's important to clarify that the test accuracy estimated through cross-validation is still not the same as the true performance on a completely unseen dataset.

Here's how cross-validation contributes to a more accurate estimate of test accuracy:

1. **Reduced Variance**: Cross-validation helps reduce the variance in performance estimation. A single train-test split can be heavily influenced by the specific data points in the test set, leading to a potentially high variance in the estimated performance. By averaging over multiple splits, cross-validation smooths out this variability and provides a more stable estimate.

2. **Better Generalization**: Cross-validation evaluates the model's performance on different subsets of the data. This means that the model has to generalize well across various data partitions to achieve good performance in cross-validation. If the model consistently performs well across all folds, it suggests better generalization.

3. **Model Selection**: Cross-validation is commonly used for model selection and hyperparameter tuning. It helps you choose the best-performing model or parameter settings based on how well they perform across different folds, which can lead to a more accurate model.

Despite these advantages, it's important to keep in mind that the estimate of test accuracy obtained through cross-validation is still based on the data you have. It doesn't guarantee that your model will perform equally well on entirely new, unseen data. There can still be issues like data distribution shifts or biases in your training data that cross-validation may not fully capture.

In practice, after using cross-validation to assess and fine-tune your model, it's essential to evaluate its performance on a truly independent and unseen test dataset, if possible. This final evaluation on a separate test set provides the most accurate estimate of how well your model will perform in real-world applications.

# ***4)*** Increasing the number of iterations during training can offer some improvement when dealing with a very small training dataset. However, it is not a comprehensive solution and comes with caveats:

1. **Overfitting Risk**: Small datasets are prone to overfitting, and increasing iterations can exacerbate this issue if not properly managed.

2. **Data Quality Matters**: The impact of more iterations depends on the quality and representativeness of the small dataset.

3. **Diminishing Returns**: There's a point of diminishing returns with more iterations; after convergence, further iterations may have limited benefits.

To address the challenges of a small dataset effectively, consider additional strategies like data augmentation, transfer learning, regularization, or, ideally, collecting more data. These approaches can help improve model performance and mitigate overfitting more reliably.